In [5]:
import numpy as np
import glob
import os
from keras.models import Model, Sequential, load_model
from keras.layers import LSTM, Bidirectional, Dense, Activation, TimeDistributed, Dropout
from keras.utils import to_categorical
from keras.optimizers import RMSprop

In [2]:
alphabet_dict = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7,
    'i': 8,
    'j': 9,
    'k': 10,
    'l': 11,
    'm': 12,
    'n': 13,
    'o': 14,
    'p': 15,
    'q': 16,
    'r': 17,
    's': 18,
    't': 19,
    'u': 20,
    'v': 21,
    'w': 22,
    'x': 23,
    'y': 24,
    'z': 25,
    ' ': 26
}

alphabet_size = len(alphabet_dict)

tone_dict = {
    (0, 0): 0,
    (0, 6): 1,
    (0, 7): 2,
    (0, 8): 3,
    (0, 9): 4,
    (1, 0): 5,
    (1, 6): 6,
    (1, 7): 7,
    (1, 8): 8,
    (2, 0): 9,
    (2, 6): 10,
    (2, 7): 11,
    (2, 8): 12,
    (3, 0): 13,
    (3, 6): 14,
    (3, 7): 15,
    (3, 8): 16,
    (4, 0): 17,
    (4, 6): 18,
    (4, 7): 19,
    (4, 8): 20,
    (5, 0): 21,
    (5, 6): 22,
    (5, 7): 23,
    (5, 8): 24
}

tone_size = len(tone_dict)

In [3]:
train_dir = 'training'
validation_dir = 'validation'

In [4]:
def batch_generator(data_dir, batch_size=32):
    while True:
        for f_path in glob.glob(os.path.join(data_dir, "*.txt")):
            with open(f_path, 'r', encoding='utf-8') as f:
                X_train = []
                y_train = []
                for line in f:
                    line = line.strip('\n')
                    char, tone = line.split(',')
                    char = list(map(int, char.split()))
                    tone = list(map(int, tone.split()))
                    X_train.append(to_categorical(char, num_classes=alphabet_size)) # one-hot encoding
                    y_train.append(to_categorical(tone, num_classes=tone_size)) # one-hot encoding
                X_train = np.array(X_train)
                y_train = np.array(y_train)
                
#                 n_samples = X_train.shape[0]
#                 if (n_samples % batch_size) != 0:
#                     repeats = batch_size - n_samples % batch_size
#                     X_last = np.repeat([X_train[-1]], repeats=repeats, axis=0)
#                     X_train = np.vstack([X_train, X_last])
#                     y_last = np.repeat([y_train[-1]], repeats=repeats, axis=0)
#                     y_train = np.vstack([y_train, y_last])
                    
                for i in range(X_train.shape[0] // batch_size):
                    yield X_train[i*batch_size:(i+1)*batch_size], y_train[i*batch_size:(i+1)*batch_size]

In [6]:
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), 
                        input_shape=(None, alphabet_size)))
model.add(Dense(tone_size, activation='softmax'))

In [6]:
# model = load_model('model.h5')

In [9]:
optimizer = RMSprop(lr=1e-4)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
model.fit_generator(batch_generator(data_dir=train_dir), steps_per_epoch=70000, epochs=1,
                    validation_data=batch_generator(data_dir=validation_dir), validation_steps=3500)

In [12]:
model.save('model.h5')